In [1]:
import os

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_community.utilities import GoogleSerperAPIWrapper

In [2]:
load_dotenv()

True

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
import pathlib
if pathlib.Path('db').exists():
    vectordb = Chroma(persist_directory='db', embedding_function=OpenAIEmbeddings())
else:
    loader = PyPDFLoader("../data/book.pdf")
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100)
    texts = splitter.split_documents(
        documents
    )
    vectordb = Chroma.from_documents(
        documents=texts,
        embedding=OpenAIEmbeddings(),
        persist_directory="db"
    )
    vectordb.persist()

In [6]:
custom_rag_prompt = """
Використай наступні **надійні** елементи, для того, щоб відповісти на питання в кінці. 
Якщо вони не містять відповіді, зверни увагу на відповідь з інтернету, хоча вона може бути не надійною. 
Якщо ти не знаєш відповіді, використаши всі свої джерела, то просто скажи про це, не потрібно вигадувати відповідь.
Використовуй не більше трьох речень, та намагайся відповісти коротко та чітко.

{context}

Відповідь з інтернету: {internet}

Питання: {question}

Корисна відповідь:"""

custom_rag_prompt = PromptTemplate.from_template(custom_rag_prompt)

In [26]:
retriever = vectordb.as_retriever()

search = GoogleSerperAPIWrapper()
def use_google_search(query):
    try:
        return search.run(query)
    except Exception as ex:
        return 'NONE'

def documents_parser(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | documents_parser, "internet" : RunnableLambda(use_google_search),  "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [27]:
type(rag_chain)

langchain_core.runnables.base.RunnableSequence

In [19]:
rag_chain.invoke("Які конституційні засади сучасної політичної системи України ви знаєте?")

'До конституційних засад сучасної політичної системи України входять демократія, принцип верховенства права, гарантії прав та свобод громадян, розділення влади на виконавчу, законодавчу та судову.'

In [16]:
stuff = search.run('Які конституційні засади сучасної політичної системи України ви знаєте?')

In [25]:
def embed_pdf(folder: str = 'data', name: str = 'book.pdf'):
    pathlib.Path(folder).mkdir(exist_ok=True)
    path = pathlib.Path(folder).joinpath(name)
    if not path.exists():
        print('Downloading book PDF.')
        gdown.download('https://drive.google.com/file/d/1CwhFM4gInp9xV4G4sdnYE_rN0StmqQ2z/view?usp=drive_link',
                       str(path), fuzzy=True)
        
embed_pdf('../data')

Downloading...
From: https://drive.google.com/uc?id=1CwhFM4gInp9xV4G4sdnYE_rN0StmqQ2z
To: C:\Users\bsvja\PycharmProjects\pdf-rag\data\book.pdf
100%|██████████| 2.37M/2.37M [00:00<00:00, 4.19MB/s]


In [29]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel


In [36]:
def flatten_input(d):
    ret = d.pop('a')
    ret.update(d)
    return ret
a = RunnableParallel(a = RunnablePassthrough(), b = RunnableLambda(lambda x: "abracadabra")) | RunnableLambda(flatten_input)

In [37]:
a.invoke({"xx" : "yy", "zz" : 11})

{'xx': 'yy', 'zz': 11, 'b': 'abracadabra'}